In [ ]:
import gym
import robo_gym
from robo_gym.wrappers.exception_handling import ExceptionHandling

In [ ]:
target_machine_ip = 'localhost' # or other machine 'xxx.xxx.xxx.xxx'

# initialize environment
env = gym.make('CubeRoomOnNavigationStack-v0', ip=target_machine_ip, gui=True)
env = ExceptionHandling(env)

env.reset(True, True)
env.state

In [1]:
import numpy as np
import copy

import robo_gym_server_modules.robot_server.client as rs_client
from robo_gym.envs.simulation_wrapper import Simulation
from robo_gym_server_modules.robot_server.grpc_msgs.python import robot_server_pb2

from robo_gym.envs.mir_nav.mir_nav import Mir100NavEnv

In [2]:
class TestSim(Simulation):
    cmd = "roslaunch task_on_nav_robot_server sim_robot_server.launch"
    def __init__(self, ip=None, lower_bound_port=None, upper_bound_port=None, gui=False, **kwargs):
        Simulation.__init__(self, self.cmd, ip, lower_bound_port, upper_bound_port, gui, **kwargs)
        rs_address=self.robot_server_ip
        self.client = rs_client.Client(rs_address)

ts = TestSim(ip='localhost', gui=True)
client = ts.client

map_size = 256

rs_state = np.zeros([(map_size**2)*2 + 17])

ignore_start = 1
map_state_len = (map_size**2)*2
ignore_len = map_state_len + 6
ignore_index = ignore_start + ignore_len

rs_state[0] = map_size
rs_state[ignore_index] = 1
rs_state[ignore_index+1] = 1
rs_state[ignore_index+2] = 10
rs_state[ignore_index+3] = 0.4
rs_state[ignore_index+4] = 0.2
rs_state[ignore_index+5] = 9.0
rs_state[ignore_index+6] = 25
rs_state[ignore_index+7] = 36
rs_state[ignore_index+8] = 0.8
rs_state[ignore_index+9] = 0.05

state_msg = robot_server_pb2.State(state=rs_state)

if not client.set_state_msg(state_msg):
    raise Exception('fuuu')

rs_state = np.array(client.get_state_msg().state)
print(rs_state)

Starting new Robot Server | Tentative 1
<class 'server_manager_pb2.RobotServer'>
True 
Successfully started Robot Server at localhost:58481
[ 2.56000000e+02 -1.00000000e+00 -1.00000000e+00 ... -4.14357424e-01
  2.15855017e-01  0.00000000e+00]


In [7]:
rs_state = np.array(client.get_state_msg().state)

In [21]:
from PIL import Image

arr = np.array(rs_state[1:1+map_size**2])
unique, counts = np.unique(arr, return_counts=True)
print(dict(zip(unique, counts)))
arr[arr == 0] = 255
arr[arr == 100] = 0
arr[arr == -1] = 128
arr = np.array(arr, dtype=np.uint8)
imgg = Image.fromarray(arr.reshape((256,256)).T, 'L')
imgg.show()

ar = np.array(rs_state[2+map_size**2:1+(map_size**2)*2+1],dtype=np.uint8)
img = Image.fromarray(ar.reshape((256,256)), 'L')
img.show()

{-1.0: 53848, 0.0: 10852, 100.0: 836}


In [ ]:
import sys
print(sys.path)

In [8]:
import numpy as np

g = np.reshape([1,1,1], (3,1))
g.flatten(), g

(array([1, 1, 1]),
 array([[1],
        [1],
        [1]]))

256.0